In [1]:
# !pip install motmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 411.2 kB/s eta 0:00:001m416.3 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m


In [2]:
#!/usr/bin/env python3
import numpy as np
import cv2
import torch
import os
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

In [3]:
#camera callibaration data of kitti
KITTI_P2_0 = np.array([[ 7.215377000000e+02, 0.000000000000e+00, 6.095593000000e+02, 4.485728000000e+01,],
                 [0.000000000000e+00, 7.215377000000e+02, 1.728540000000e+02, 2.163791000000e-01,],
                 [0.000000000000e+00, 0.000000000000e+00, 1.000000000000e+00, 2.745884000000e-03],])
                
KITTI_P3_0 = np.array([[ 7.215377000000e+02, 0.000000000000e+00, 6.095593000000e+02, -3.395242000000e+02,],
                 [ 0.000000000000e+00, 7.215377000000e+02, 1.728540000000e+02, 2.199936000000e+00,],
                 [ 0.000000000000e+00, 0.000000000000e+00, 1.000000000000e+00, 2.729905000000e-03],])


In [19]:
from pipeline import Pipeline

In [20]:
pp = Pipeline(max_age=1, min_hits=3, iou_threshold=0.1, no_of_future_frames=6, no_of_past_frames=6)

Starting Pipeline.
Reading Pipeline params.
Loading Pipeline.
Starting Yolo3DNode.
Reading params.
Loading model.
Done loading model.
--------------------- trajectory ---------------------
self.max_age =  1 
 self.min_hits =  3 
 self.iou_threshold =  0.1 
 self.no_of_future_frames =  6 
 self.no_of_past_frames =  6 

Starting View.
Reading View params.


In [21]:
# path = 'kitti_s1' #836
path = 'kitti_s2' #77
# path = 'kitti_s3' #446
# path = 'kitti_s4' #313
# path = 'kitti_s5' #21
# path = 'kitti_s6' #432

In [14]:
pwd

'/home/chirathv97/FYP/Stereo-Vision-Based-Vehicle-Forward-Collision-Warning-System'

In [22]:
length = 78
start_track = time.time()

current_bb_list=[]
future_bb_list=[]

for frame in range (length):
    print('frame',frame)
    left_image = cv2.imread("./data/"+path+"_left/%06d" % frame+".png")
    right_image = cv2.imread("./data/"+path+"_right/%06d" % frame+".png")

    rgb_image, track_bbs_ids=pp.assignTrackID(left_image,right_image)


    for bbs in track_bbs_ids: #for each vehicle in a image frame
        bb=bbs[0] #current bounding box
        future_bbs=bbs[1] #future bounding boxes
        vehicle_ID=bbs[-1] #vehicle id
        
        if len(future_bbs) > 0:
            future_bb=future_bbs[0]
            current_bb_list.append(bb)
            future_bb_list.append(future_bb)

            
print('len(current_bb_list)',len(current_bb_list))
print('len(future_bb_list)',len(future_bb_list))

frame 0
frame 1


/home/chirathv97/FYP/Stereo-Vision-Based-Vehicle-Forward-Collision-Warning-System/visualDet3D/networks/lib/PSM_cost_volume.py:87: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  volatile= not self.training
/home/chirathv97/FYP/Stereo-Vision-Based-Vehicle-Forward-Collision-Warning-System/visualDet3D/networks/lib/PSM_cost_volume.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  volatile= not self.training
/home/chirathv97/FYP/Stereo-Vision-Based-Vehicle-Forward-Collision-Warning-System/trajectory/trajectory.py:360: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ret)


frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20
frame 21
frame 22
frame 23
frame 24
frame 25
frame 26
frame 27
frame 28
frame 29
frame 30
frame 31
frame 32
frame 33
frame 34
frame 35
frame 36
frame 37
frame 38
frame 39
frame 40
frame 41
frame 42
frame 43
frame 44
frame 45
frame 46
frame 47
frame 48
frame 49
frame 50
frame 51
frame 52
frame 53
frame 54
frame 55
frame 56
frame 57
frame 58
frame 59
frame 60
frame 61
frame 62
frame 63
frame 64
frame 65
frame 66
frame 67
frame 68
frame 69
frame 70
frame 71
frame 72
frame 73
frame 74
frame 75
frame 76
frame 77
len(current_bb_list) 130
len(future_bb_list) 130


In [25]:
gt_bboxes_list=current_bb_list
pred_bboxes_list=future_bb_list

In [26]:
import motmetrics as mm

# Create an accumulator to store tracking event data
acc = mm.MOTAccumulator()

# Iterate through each frame and bounding box predictions
for frame_number, (gt_bboxes, pred_bboxes) in enumerate(zip(gt_bboxes_list, pred_bboxes_list)):
    # Update the accumulator with the frame's ground truth and predicted bounding boxes
    acc.update(gt_bboxes, pred_bboxes)

# Compute the metrics using the accumulator
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['num_false_positives', 'num_misses', 'num_switches', 'num_objects'], name='MOTA')

# Calculate the MOTA score
MOTA = 1 - summary['num_false_positives'] / summary['num_objects'] - summary['num_misses'] / summary['num_objects'] - summary['num_switches'] / summary['num_objects']

# Print the MOTA score
print("summary['num_false_positives']:", summary['num_false_positives'])
# Print the MOTA score
print("summary['num_objects']:", summary['num_objects'])
# Print the MOTA score
print("summary['num_misses']:", summary['num_misses'])
# Print the MOTA score
print("summary['num_objects']", summary['num_objects'])
# Print the MOTA score
print("MOTA score:", MOTA)


TypeError: update() missing 1 required positional argument: 'dists'